In [ ]:
# def degToCompass(num):
#     val=int((num/22.5)+.5)
#     arr=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
#     return (arr[(val % 16)])

def degToCompass(num):
    val=int((num/45)+.5)
    arr=["N","NE","E","SE", "S","SW","W","NW"]
    return (arr[(val % 8)])

import flask
from flask_cors import CORS
from flask import jsonify

app = flask.Flask(__name__)
CORS(app)
app.config["DEBUG"] = False

import os
import glob

import pandas as pd
import numpy as np
from subprocess import Popen, PIPE

def get_gp2_data(nb_rows=30,main_dir = '../../Sa-modat-mpo-pr/Data-Safi/'):
    # Find last File
    p = Popen("last_file_gp2.bat", shell=True, stdout=PIPE,)
    stdout, stderr = p.communicate()
    file_path = stdout.decode('utf-8').rstrip()
    
    # Load last File
    data = pd.read_csv(main_dir + file_path,low_memory=False,
                   delimiter='\t',quotechar='"',decimal=',',).dropna()

    # Rename Columns and filter on last 30 min
    data = data.rename(columns={'Unnamed: 0' : 'datetime',
                                'Speed@1m': 'Wind Speed (m.s-1)', 
                                'Dir': 'Wind Direction (deg C)',
                                'AirTemp' : 'Air Temperature (deg C)',
                                "Rad'n" : 'Radiation (W.m-2)',
                                'Rain@1m' : 'Precipitation (mm)'}).tail(nb_rows)
    # Format Day & Time
    data['Day'] = data.datetime.map(lambda x : x[0:10])
    data['Day'] = data['Day'].map(lambda x : x.split('/')[2] + '-' + x.split('/')[1] + '-'+ x.split('/')[0])
    data['Time'] = data.datetime.map(lambda x : x[11:16])
    
    data['Wind Direction'] = data['Wind Direction (deg C)'].map(lambda x : degToCompass(float(x.replace(',','.'))))
    
    # Filter Columns
    data.drop(columns={'datetime','Pressure','RH','Power'},inplace=True)
    
    return data

def get_akabar_data(nb_rows=30,main_dir = '../../Sa-modat-mpo-pr/AKABAR/'):
    # Find last File
    p = Popen("last_file_akabar.bat", shell=True, stdout=PIPE,)
    stdout, stderr = p.communicate()
    file_path = stdout.decode('utf-8').rstrip()

    data = pd.read_csv(main_dir + file_path,low_memory=False,
                           delimiter='\t',quotechar='"',decimal=',')

    # drop 2 rows (contains unit)
    data = data.loc[2:,:]
    data = data.rename(columns={'AKABAR': 'datetime',
                             'Solar Rad (wat/m2)': 'Radiation (W.m-2)',
                             'Temperature (*C)' : 'Air Temperature (deg C)',
                             'Rainfall (mm)' : 'Precipitation (mm)',
                             'Wind Dir (Deg)' : 'Wind Direction (deg C)',
                             'Wind Speed (km/h)' : 'Wind Speed (m.s-1)'}).tail(nb_rows)

    # Format Day & Time
    data['Day'] = data.datetime.map(lambda x : x[0:10])
    data['Time'] = data.datetime.map(lambda x : x[11:16])

    data['Wind Direction'] = data['Wind Direction (deg C)'].map(lambda x : degToCompass(float(x.replace(',','.'))))

    # Filter Columns
    data.drop(columns={'datetime','RH (%)','Wind Gust (km/h)','Dew Point (*C)'},inplace=True)
    
    return data

@app.route('/data/akabar_last_row/', methods=['GET'])
def akabar_last_row():
    return jsonify(get_akabar_data(nb_rows=1).to_dict(orient='records'))

@app.route('/data/akabar_last_30min/', methods=['GET'])
def akabar_last_30min():
    return jsonify(get_akabar_data(nb_rows=30).to_dict(orient='records'))

@app.route('/data/gp2_last_row/', methods=['GET'])
def gp2_last_row():
    return jsonify(get_gp2_data(nb_rows=1).to_dict(orient='records'))

@app.route('/data/gp2_last_30min/', methods=['GET'])
def gp2_last_30min():
    return jsonify(get_gp2_data(nb_rows=30).to_dict(orient='records'))

app.run(port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Aug/2021 16:04:12] "GET /data/gp2_last_30min/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:04:12] "GET /data/gp2_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:17:00] "GET /data/akabar_last_30min/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:17:00] "GET /data/akabar_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:23] "GET /data/gp2_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:23] "GET /data/gp2_last_30min/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:28] "GET /data/akabar_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:28] "GET /data/akabar_last_30min/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:44] "GET /data/gp2_last_30min/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:18:44] "GET /data/gp2_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:19:24] "GET /data/akabar_last_row/ HTTP/1.1" 200 -
127.0.0.1 - - [02/Aug/2021 16:19:24] "GET /data/akabar_la